In [1]:
import pandas as pd
from tensorflow import keras
import numpy as np
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import string
from keras import layers
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing import sequence
from tensorflow.keras.optimizers import RMSprop
from collections import Counter

In [2]:
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
train = pd.read_csv(train_file_path, sep='\t', header=None)
test = pd.read_csv(test_file_path, sep='\t', header=None)

In [4]:
# Preprocessing
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

In [5]:
# remove stopwords
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

In [6]:
train[1] = train[1].map(remove_URL) # map(lambda x: remove_URL(x))
train[1] = train[1].map(remove_punct)
train[1] = train[1].map(remove_stopwords)

In [7]:
test[1] = test[1].map(remove_URL) # map(lambda x: remove_URL(x))
test[1] = test[1].map(remove_punct)
test[1] = test[1].map(remove_stopwords)

In [8]:
# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

df = pd.concat([train, test], ignore_index=True)
counter = counter_word(df[1])
num_unique_words = len(counter)

In [9]:
X_train = train[1].to_numpy()
X_test = test[1].to_numpy()
y_train = train[0]
y_test = test[0]
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [10]:
max_len = 20
# create tokenizer
token = Tokenizer(num_words = num_unique_words)
token.fit_on_texts(X_train)
# tokenize X_train and X_test
X_train_seq = token.texts_to_sequences(X_train)
X_test_seq = token.texts_to_sequences(X_test)
# pad X_train and X_test
X_train_pad = sequence.pad_sequences(X_train_seq , maxlen = max_len)
X_test_pad = sequence.pad_sequences(X_test_seq , maxlen = max_len)

In [12]:
model = Sequential([
    layers.Embedding(num_unique_words , 32 , input_length = max_len),
    layers.LSTM(64),
    layers.Flatten(),
    layers.Dense(250, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(120, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])

In [13]:
model.compile(loss = 'binary_crossentropy' , optimizer = 'RMSprop' , metrics = 'accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 32)            302656    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 250)               16250     
                                                                 
 dropout (Dropout)           (None, 250)               0         
                                                                 
 dense_1 (Dense)             (None, 120)               30120     
                                                                 
 dropout_1 (Dropout)         (None, 120)               0

In [88]:
model.fit(X_train_pad,y_train,batch_size=128,epochs=10,
          validation_data=(X_test_pad, y_test))

Epoch 1/10
33/33 [==============================] - 1s 19ms/step - loss: 3.8827e-06 - accuracy: 1.0000 - val_loss: 0.1415 - val_accuracy: 0.9820
Epoch 2/10
33/33 [==============================] - 1s 19ms/step - loss: 1.1216e-06 - accuracy: 1.0000 - val_loss: 0.1470 - val_accuracy: 0.9828
Epoch 3/10
33/33 [==============================] - 1s 19ms/step - loss: 2.4117e-06 - accuracy: 1.0000 - val_loss: 0.1544 - val_accuracy: 0.9828
Epoch 4/10
33/33 [==============================] - 1s 16ms/step - loss: 2.5700e-07 - accuracy: 1.0000 - val_loss: 0.1835 - val_accuracy: 0.9828
Epoch 5/10
33/33 [==============================] - 1s 16ms/step - loss: 1.3769e-07 - accuracy: 1.0000 - val_loss: 0.1769 - val_accuracy: 0.9820
Epoch 6/10
33/33 [==============================] - 1s 16ms/step - loss: 2.9687e-08 - accuracy: 1.0000 - val_loss: 0.2262 - val_accuracy: 0.9820
Epoch 7/10
33/33 [==============================] - 1s 16ms/step - loss: 2.3168e-08 - accuracy: 1.0000 - val_loss: 0.2217 - val_ac

In [106]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # preprocessing
  pred_text = remove_URL(pred_text)
  pred_text = remove_punct(pred_text)
  pred_text = remove_stopwords(pred_text)
  pred_text = np.array([pred_text])
  pred_text = token.texts_to_sequences(pred_text)
  pred_text = sequence.pad_sequences(pred_text , maxlen = max_len)
  # predicting on data
  predict = model.predict(pred_text)
  prediction = [predict.item(), 'ham' if predict.item() < 0.5 else 'spam']
  return (prediction)

pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

[0.5930676460266113, 'spam']


In [105]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!


In [108]:
model.save('spam_clas.h5')